In [ ]:
## Animal Tracking Visualisations
# Visualisations to assist security forces in the fight against poaching

# Import relevant libraries 
import pandas as pd
import datetime as dt
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon

# Import animal tracking data
# URL of the data set
url = "/Users/samwinkworth/final-project/data/forestObservatory.xlsx"

# Read first page of the data set and alter column names to remove whitespace
forestObs = pd.read_excel(url, 'Catalog')
forestObs = forestObs.rename(columns={'Sheet_Name ': 'Sheet_Name', 'Animal Type ': 'Animal_Type'})

# Alter animal names so they're consistent
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace("-","_")
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace(" ","")
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace("ElephantorLeapardcat","Elephant_or_Leapard_cat")
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace("?","", regex=True)
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace("(","_", regex=True)
forestObs['Animal_Type'] = forestObs['Animal_Type'].str.replace(")","", regex=True)

# Create a list of all the sheets and animals in the data set
#dname = forestObs['Sheet_Name'].to_list()
#animal_name = forestObs['Animal_Type']

# Combine the two lists
animalDoc = list(zip(forestObs.Animal_Type, forestObs.Sheet_Name))

# Hide outputs but saves output 
#%%capture

sheet_list = ['Kuma_GPSloc_27Apr-1Nov13',
 'Jasmin_GPSloc_26Oct11-19Apr12',
 'Putut_GPSloc_26Oct11-16Aug12',
 'Bikang1_GPSloc_20Jan-15Aug13',
 'Bikang2_GPSloc_23Jan-15Aug13',
 'Sejati_GPSloc_5Jun-29Oct13',
 'Umas2_GPSloc_17Apr-24Dec14',
 'Ita_GPSloc_4Jun13-24Jul14r',
 'Abaw_GPSloc_23Feb14-21Jul15',
 'Liun_GPSloc_15Apr11-2Nov13',
 'Dara_GPSloc_26Feb13-22Mar15',
 'Jasper_GPSloc_16Mar14-12Aug15',
 'Guli_GPSloc_20Jun15-6Feb17',
 'Maliau_GPSloc_8Jul15-18Jun17',
 'Tunglap_GPSloc_8Dec13-17Nov15',
 'Seri_GPSloc_8Nov15-10Jan18',
 'WinWin_GPSloc_18Dec13-28Jun16',
 'Puteri_GPSloc_20Oct12-26Jul16',
 'Tulid_GPSloc_7Dec13-20May16',
 'Kasih_GPSloc_6Jul14-20May17',
 'Aqeela_GPSloc_5Oct10-6Jul13',
 'Merotai_GPSloc_18Apr15-30Sep16']

df = pd.concat(pd.read_excel(url, sheet_name=sheet_list), ignore_index=True)

# Keep only necessary columns
df = df[['GMT Date','GMT Time', 'Lat', 'Lon', 'Alt (m)', 'Type', 'Name', 'Age', 'Sex', 'Capture_Loc', 'Release_Loc', 'Release_Date']]

In [ ]:
df = df.drop_duplicates()

df.isnull().sum()

In [ ]:
# Drop rows containing impossible dates 
i_date = df[((df['GMT Date'] == '2014-00-15'))].index
i_date1= df[((df['GMT Date'] == '2015-06-00'))].index
df = df.drop(i_date)
df = df.drop(i_date1)

# Create new column with date format that can be passed through timeStampedGeoJson function
df['GMT Date ISO'] = pd.to_datetime(df['GMT Date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

In [ ]:
# Method to remove coordinates outside of Borneo
# Define coordinates of polygon surrounding Borneo
coords = [[109.9099731, 2.1926104], [117.0153809, 7.5694373], [119.4873047, 5.4410223], 
          [119.2016602, 0.8788718], [116.3232422, -4.28068], [111.862793, -3.973861],
          [109.2480469, -2.6577378], [108.1494141, 1.6257584], [109.9099731, 2.1926104]]

# Turn coordinates into Shapely polgon
poly = Polygon(coords)

# Create dataframe with geometry of defined polygon
poly_geom= gpd.GeoDataFrame()
poly_geom['geometry'] = None
poly_geom.loc[0,'geometry'] = poly

# Create list of geometry points
geom_data = [Point(xy) for xy in zip(df.Lon, df.Lat)]
gpd_df = gpd.GeoDataFrame(df, geometry=geom_data)

# Create new data frame by using spatial join and exclude any rows with coordinates outside of polygon
df0 = gpd.sjoin(gpd_df, poly_geom)

In [ ]:
# Export CSV of all processed tracking data
df0.to_csv('/Users/samwinkworth/final-project/data/animal_tracking.csv', index=False)